In [61]:
import pandas as pd
import requests

COURSE_ID = "223125"
API_KEY = "ENTER YOUR API KEY HERE"

In [63]:
class Extractor:
    def __init__(self, course_id):
        self.course_id = course_id

    def getEverything(self):
        URL = f"https://q.utoronto.ca/api/v1/courses/{self.course_id}/quizzes"
        headers = {
            "Content-Type": "application/json; charset=utf-8", 
            "Authorization": f"Bearer {API_KEY}"
            }
        r = requests.get(url = URL, headers=headers)
        quiz_ids = []
        allQuizzes = r.json()
        for aQuiz in allQuizzes:
            quiz_ids.append(aQuiz['id'])

        # Get everything for every quiz.
        quiz_id_list = []
        user_id_list = []
        attempt_list = []
        first_viewed_time_list = []
        last_viewed_time_list = []
        score_list = []
        kept_score_list = []

        for quiz_id in quiz_ids:
            URL = f"https://q.utoronto.ca/api/v1/courses/{self.course_id}/quizzes/{quiz_id}/submissions"
            headers = {
                "Content-Type": "application/json; charset=utf-8", 
                "Authorization": f"Bearer {API_KEY}"
                }
            r = requests.get(url = URL, headers=headers)
            allSubmissions = r.json()['quiz_submissions']
            for submission in allSubmissions:
                submission_id = submission['id']
                URL = f"https://q.utoronto.ca/api/v1/courses/{self.course_id}/quizzes/{quiz_id}/submissions/{submission_id}"
                headers = {
                    "Content-Type": "application/json; charset=utf-8", 
                    "Authorization": f"Bearer {API_KEY}"
                    }
                r = requests.get(url = URL, headers=headers)
                data = r.json()

                assert len(data['quiz_submissions']) == 1
                attempt_numbers = data['quiz_submissions'][0]['attempt']           
                for attempt in range(1, attempt_numbers + 1):
                    URL = f"https://q.utoronto.ca/api/v1/courses/{self.course_id}/quizzes/{quiz_id}/submissions/{submission_id}?attempt={str(attempt)}"
                    headers = {
                        "Content-Type": "application/json; charset=utf-8", 
                        "Authorization": f"Bearer {API_KEY}"
                        }
                    r = requests.get(url = URL, headers=headers)
                    data = r.json()['quiz_submissions'][0]
                    quiz_id_list.append(quiz_id)
                    user_id_list.append(data['user_id'])
                    attempt_list.append(attempt)
                    first_viewed_time_list.append(data['started_at'])
                    last_viewed_time_list.append(data['finished_at'])
                    score_list.append(data['score'])
                    kept_score_list.append(data['kept_score'])

        df = pd.DataFrame(
            data = {
                'quiz_id': quiz_id_list,
                'user_id': user_id_list,
                'attempt': attempt_list,
                'started_at': first_viewed_time_list,
                'finished_at': last_viewed_time_list,
                'score': score_list,
                'kept_score': kept_score_list
            }
        )

        df.to_csv(f"{COURSE_ID}_quiz_stats.csv", index = False)


    # def extractFirstLastViewedTimesAndScore(self, quiz_id, submission_id, attempt = 1):
    #     # https://canvas.instructure.com/doc/api/quiz_submission_events.html
    #     URL = f"https://q.utoronto.ca/api/v1/courses/{self.course_id}/quizzes/{quiz_id}/submissions/{submission_id}/events/?attempt={attempt}"
    #     headers = {
    #         "Content-Type": "application/json; charset=utf-8", 
    #         "Authorization": f"Bearer {API_KEY}"
    #         }
    #     r = requests.get(url = URL, headers=headers)
    #     data = r.json()

    #     quiz_submission_events = data['quiz_submission_events']
    #     first_viewed_time = None
    #     last_viewed_time = None
    #     score = None
    #     for quiz_submission_event in quiz_submission_events:
    #         if quiz_submission_event['event_type'] == "submission_created":
    #             first_viewed_time = quiz_submission_event['created_at']
    #             break
    #     sorted_events_by_created_at = sorted(quiz_submission_events, key=lambda d: parse(d['created_at']), reverse=True)

    #     last_viewed_time = sorted_events_by_created_at[0]['created_at']
    #     return (first_viewed_time, last_viewed_time, score)

Example of how to use

In [64]:
extractor = Extractor(COURSE_ID)
extractor.getEverything()
#extractor.extractFirstLastViewedTimesAndScore('195052', '10657535')

TypeError: string indices must be integers